# Saving Models

At the moment I am trying to work out how to save models etc. This is a space where I'll be wrapping my head around how this works.

In [1]:
import tensorflow as tf

In [2]:
# Create some variables.
v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
    sess.run(init_op)
    # Do some work with the model.
    inc_v1.op.run()
    dec_v2.op.run()
    # Save the variables to disk.
    save_path = saver.save(sess, "./tmp/model.ckpt")
    print("Model saved in file: %s" % save_path)

Model saved in file: ./tmp/model.ckpt


In [3]:
tf.get_default_graph().get_operations()

[<tf.Operation 'v1/Initializer/zeros' type=Const>,
 <tf.Operation 'v1' type=VariableV2>,
 <tf.Operation 'v1/Assign' type=Assign>,
 <tf.Operation 'v1/read' type=Identity>,
 <tf.Operation 'v2/Initializer/zeros' type=Const>,
 <tf.Operation 'v2' type=VariableV2>,
 <tf.Operation 'v2/Assign' type=Assign>,
 <tf.Operation 'v2/read' type=Identity>,
 <tf.Operation 'add/y' type=Const>,
 <tf.Operation 'add' type=Add>,
 <tf.Operation 'Assign' type=Assign>,
 <tf.Operation 'sub/y' type=Const>,
 <tf.Operation 'sub' type=Sub>,
 <tf.Operation 'Assign_1' type=Assign>,
 <tf.Operation 'init' type=NoOp>,
 <tf.Operation 'save/Const' type=Const>,
 <tf.Operation 'save/SaveV2/tensor_names' type=Const>,
 <tf.Operation 'save/SaveV2/shape_and_slices' type=Const>,
 <tf.Operation 'save/SaveV2' type=SaveV2>,
 <tf.Operation 'save/control_dependency' type=Identity>,
 <tf.Operation 'save/RestoreV2/tensor_names' type=Const>,
 <tf.Operation 'save/RestoreV2/shape_and_slices' type=Const>,
 <tf.Operation 'save/RestoreV2' typ

In [ ]:
from keras import layers

FILTERS = 256

def residual_tower(X, num_blocks = 10, filters = FILTERS):
    '''
    Constructs a residual tower consisting of a convolutional block, followed
    by a sequence of `num_blocks` residual blocks, each of with a fixed number of
    filters.
    
    The residual tower output is passed to the policy + value heads for
    move prediction and board evaluation respectively'''
    
    def convolutional_block(X, filters = FILTERS, kernel_size = (3, 3), activation = True):
        '''
        The convolutional block for the network. Performs convolution with a given number
        of filters and kernel size.

        If an activation is given, then it will be applied to the output before the
        result is returned. If you want no activation function, then use activation = None'''

        # run y through convolutional layer
        y = layers.Conv2D(
                filters = filters, 
                kernel_size = kernel_size,
                data_format = 'channels_first',
                padding='same')(X)

        # Apply batch normalisation
        y = tf.layers.batch_normalization(y)
        if activation:
            # Apply activation function
            y = tf.nn.relu(y)

        # return output tensor
        return y
    
    def residual_block(X, activation = 'relu'):
        '''
        '''
        y = convolutional_block(X)
        y = convolutional_block(y, activation = None)
        X = convolutional_block(X, kernel_size = (1, 1), activation = None)
        y = tf.nn.relu(X + y)
        return y
    
    ## Construct Residual Tower:
    y = convolutional_block(X)
    for i in range(num_blocks):
        y = residual_block(y)
        
    def policy_head(y):
        '''
        '''
        y = convolutional_block(y, activation = True)
        y = layers.Flatten()(y)
        y = layers.Dense(units = 5000)(y)
        y = layers.Activation('softmax')(y)
        return y

    def value_head(y):
        '''
        '''
        y = convolutional_block(y)
        y = layers.Flatten()(y)
        y = layers.Dense(units = FILTERS, activation = 'relu')(y)
        y = layers.Dense(units = 1, activation = 'tanh')(y)
        return y
    
    return policy_head(y), value_head(y)